In [ ]:
# import tqdm
# import torch as T
# import os

# pth = "./dataset_embeddings/convnextv2_atto.fcmae_ft_in1k"
# num_encoding_blocks = len(os.listdir(pth))

# embeddings = T.zeros(num_encoding_blocks * 250, 320, 7, 7)
# valid = T.zeros(num_encoding_blocks * 250, dtype=T.bool)

# print(f"number of encodings: {len(os.listdir(pth))}")
# for (i, f) in tqdm.tqdm(enumerate(os.listdir(pth)), total=num_encoding_blocks, desc="Loading encodings"):
#     ten = T.load(os.path.join(pth, f), map_location="cpu")
#     block_size = ten.size(0)
#     embeddings[(i * 250):(i * 250 + block_size), ...] = ten[:]
#     valid[(i * 250):(i * 250 + block_size)] = True

# T.save(embeddings, "./dataset_embeddings/convnextv2_atto.fcmae_ft_in1k-embeddings.pt")
# T.save(valid,      "./dataset_embeddings/convnextv2_atto.fcmae_ft_in1k-valid.pt")

In [ ]:
# import timm
# import torch as T
# import torch.nn as nn
# import pytorch_lightning as pl
# from motion_capture.model.models import VQVAE, VQVAEHead

# class VisionModel(pl.LightningModule):
    
#     def __init__(
#         self,
        
#         backbone: str,
#         vqvae: dict,
#         heads: dict,
        
#         # - training parameters
#         optimizer: T.optim.Optimizer = None,
#         optimizer_kwargs: dict = None,
#         lr_scheduler_warmup_epochs: int = None,
#         lr_scheduler: T.optim.lr_scheduler = None,
#         lr_scheduler_kwargs: dict = None
        
#         # - loss scale parameters
#         reconstruction_loss_scale: float = 1.0,
#         codebook_loss_scale: float = 1.0,
#         prediction_loss_scales: dict = None
#         ):
        
#         super().__init__()
#         self.save_hyperparameters()
        
#         self.backbone = timm.create_model(backbone, pretrained=False, features_only=True)
#         self.vqvae = vqvae = VQVAE(**vqvae)
        
#         self.heads = nn.ModuleDict({k: VQVAEHead(**v) for k, v in heads.items()})
        
#     def forward(self, x, skip_backbone=False):
        
#         if not skip_backbone:
#             backbone_out = self.backbone(x)[-1].flatten(2).permute(0, 2, 1)
#         else:
#             backbone_out = x
        
#         vqvae_out = self.vqvae(backbone_out)
        
#         heads_out = {}
#         for k in self.heads:
#             heads_out[k] = self.heads[k](vqvae_out["codebook_onehots"])
        
#         return {
#             "heads": heads_out, 
#             "vqvae": vqvae_out
#         }
    
#     def get_losses(self, 
#         heads_out, heads_targets,
#         vqvae_out, vqvae_target):
        
#         reconstruction_loss, codebook_loss = self.vqvae.compute_loss(vqvae_reconstruction_target, vqvae_out["reconstruction"], vqvae_out["z"], vqvae_out["codebook_indecies"])
#         prediction_losses = {
#             k: self.hparams.prediction_loss_scales.get(k, 1) * self.heads[k].compute_loss(heads_targets[k], heads_out[k]) 
#             for k in self.heads
#         }
        
#         return {
#             "vqvae-reconstruction": reconstruction_loss * self.hparams.reconstruction_loss_scale,
#             "vqvae-codebook": codebook_loss * self.hparams.codebook_loss_scale,
#             **prediction_losses
#         }
    
#     # def head_wise_step(self, batch, mode):
        
#     #     x, y = batch
#     #     outputs = self(x)
#     #     losses = {}
#     #     for k, head_output in outputs.items():
#     #         loss = self.compute_loss(head_output, y[k], self.hparams.heads[k]["loss_fn"])
#     #         losses[k] = loss
#     #         self.log(f"{mode}_loss_{k}", loss, on_step=True, on_epoch=False, prog_bar=True)
#     #     self.log(f"{mode}_loss", sum(losses.values()) / len(losses), on_step=False, on_epoch=True, prog_bar=True)
#     #     return losses
    
#     # def training_step(self, batch, batch_id):
#     #     return self.head_wise_step(batch, "train")
#     # def validation_step(self, batch, batch_idx):
#     #     self.log("lr", self.trainer.optimizers[0].param_groups[0]["lr"], on_step=False, on_epoch=True, prog_bar=False)
#     #     return self.head_wise_step(batch, "val")
#     # def test_step(self, batch, batch_idx):
#     #     return self.head_wise_step(batch, "test")
    
#     # def configure_optimizers(self):
        
#     #     assert self.hparams.optimizer, "optimizer not set for training"
        
#     #     opt = self.hparams.optimizer(self.parameters(), **self.hparams.optimizer_kwargs)
        
#     #     warmup_scheduler = T.optim.lr_scheduler.LinearLR(opt, start_factor=0.1, total_iters=self.hparams.lr_scheduler_warmup_epochs)
#     #     scheduler = self.hparams.lr_scheduler(opt, **self.hparams.lr_scheduler_kwargs)
#     #     lr_scheduler = T.optim.lr_scheduler.SequentialLR(opt, schedulers=[
#     #         warmup_scheduler, 
#     #         scheduler
#     #     ], milestones=[self.hparams.lr_scheduler_warmup_epochs])
        
#     #     return {
#     #         "optimizer": opt,
#     #         "lr_scheduler": lr_scheduler
#     #     }

In [ ]:
# model = VisionModel(
#     backbone = "timm/convnextv2_atto.fcmae_ft_in1k",
#     vqvae = {
#         "input_dim": 320,
#         "codebook_dim": 512,
#         "num_codebook_entries": 1024,
#         "output_dim": 320,
#         "codebook_sequence_length": 15,
#         "output_sequence_length": 49,
#         "transformer": {
#             "nhead": 2,
#             "num_encoder_layers": 1,
#             "num_decoder_layers": 1,
#             "dim_feedforward": 512,
#             "dropout": 0.1,
#             "activation": "gelu"
#         }
#     },
#     heads = {
#         "personBbox": {
#             "input_dim": 1024,
#             "output_dim": 4,
#             "latent_dim": 512,
#             "output_sequence_length": 10,
#             "output_type": "continuous",
#             "transformer": {
#                 "nhead": 2,
#                 "num_encoder_layers": 1,
#                 "num_decoder_layers": 1,
#                 "dim_feedforward": 512,
#                 "activation": "gelu"
#             }
#         }
#     }
    
# )

In [ ]:
# from motion_capture.core import benchmark

# benchmark.model_speedtest(model, (3, 3, 224, 224))


In [ ]:
student_net_dataset = []
teacher_net_dataset = []

from motion_capture.data.datasets import COCO2017GlobalPersonInstanceSegmentation
person_instance_dataset = COCO2017GlobalPersonInstanceSegmentation(
    image_folder_path = "//192.168.2.206/data/datasets/COCO2017/images",
    annotation_folder_path = "//192.168.2.206/data/datasets/COCO2017/annotations",
    image_shape_WH=(224, 224),
    max_num_persons=10,
    max_segmentation_points=100
)

## PyTorch Pretrained Models Speedtest

In [ ]:
from torchvision import models as torchModels
from motion_capture.model import models as mocapModels
import torch as T

In [ ]:
object_detection_models = {
    "fcos_resnet50_fpn": torchModels.detection.fcos_resnet50_fpn(weights=torchModels.detection.FCOS_ResNet50_FPN_Weights.COCO_V1),
    "fasterrcnn_mobilenet_v3_large_320_fpn": torchModels.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights=torchModels.detection.FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.COCO_V1),
    "fasterrcnn_mobilenet_v3_large_fpn": torchModels.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=torchModels.detection.FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1),
    "fasterrcnn_resnet50_fpn_v2": torchModels.detection.fasterrcnn_resnet50_fpn_v2(weights=torchModels.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1),
    "fasterrcnn_resnet50_fpn": torchModels.detection.fasterrcnn_resnet50_fpn(weights=torchModels.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1),
    "retinanet_resnet50_fpn_v2": torchModels.detection.retinanet_resnet50_fpn_v2(weights=torchModels.detection.RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1),
    "retinanet_resnet50_fpn": torchModels.detection.retinanet_resnet50_fpn(weights=torchModels.detection.RetinaNet_ResNet50_FPN_Weights.COCO_V1),
    "ssd300_vgg16": torchModels.detection.ssd300_vgg16(weights=torchModels.detection.SSD300_VGG16_Weights.COCO_V1),
    "ssdlite320_mobilenet_v3_large": torchModels.detection.ssdlite320_mobilenet_v3_large(weights=torchModels.detection.SSDLite320_MobileNet_V3_Large_Weights.COCO_V1),
}

torch.Size([1, 512, 200])

In [ ]:
backbone_models = [
    # "deeplabv3_mobilenet_v3_large",
    # "deeplabv3_resnet101",
    # "deeplabv3_resnet50",
    "fasterrcnn_mobilenet_v3_large_320_fpn",
    "fasterrcnn_mobilenet_v3_large_fpn",
    "fasterrcnn_resnet50_fpn",
    "fasterrcnn_resnet50_fpn_v2",
    # "fcn_resnet101",
    # "fcn_resnet50",
    "fcos_resnet50_fpn",
    "keypointrcnn_resnet50_fpn",
    # "lraspp_mobilenet_v3_large",
    "maskrcnn_resnet50_fpn",
    "maskrcnn_resnet50_fpn_v2",
    "retinanet_resnet50_fpn",
    "retinanet_resnet50_fpn_v2",
]

In [ ]:
from motion_capture.core import benchmark

for model in backbone_models:
    print(f"benchmarking {model}")
    model = torchModels.get_model(model).backbone
    benchmark.model_speedtest(model, (1, 3, 224, 224), ntests=100)

NameError: name 'backbone_models' is not defined

In [ ]:
for model in backbone_models:
    backbone_out = torchModels.get_model(model).backbone(T.randn(1, 3, 448, 448))
    last_3_shapes = [backbone_out[a].shape for a in list(backbone_out)[-3:]]
    print(f"model: {model}: {last_3_shapes}")

NameError: name 'backbone_models' is not defined

In [ ]:
uca = mocapModels.UpsampleCrossAttentionNetwork(
    backbone=object_detection_models["fasterrcnn_mobilenet_v3_large_320_fpn"].backbone,
    neck=mocapModels.UpsampleCrossAttentionrNeck(128, 256, 1),
    heads={
        "bboxes": mocapModels.SelfAttentionHead(128, 4, 10, 256)
    }
)

backbone_out = uca.backbone(T.randn(1, 3, 224, 224))
print([v.shape for v in list(backbone_out.values())])

[torch.Size([1, 256, 7, 7]), torch.Size([1, 256, 7, 7]), torch.Size([1, 256, 4, 4])]


In [ ]:
segmentation_models = {
    "fcn_resnet50": models.segmentation.fcn_resnet50(weights=models.segmentation.FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1),
    "fcn_resnet101": models.segmentation.fcn_resnet101(weights=models.segmentation.FCN_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_resnet50": models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_resnet101": models.segmentation.deeplabv3_resnet101(weights=models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_mobilenet_v3_large": models.segmentation.deeplabv3_mobilenet_v3_large(weights=models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1),
    "lraspp_mobilenet_v3_large": models.segmentation.lraspp_mobilenet_v3_large(weights=models.segmentation.LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1),
}

instance_segmentation_models = {
    "maskrcnn_resnet50_fpn": models.detection.maskrcnn_resnet50_fpn(weights=models.detection.MaskRCNN_ResNet50_FPN_Weights.COCO_V1),
    "maskrcnn_resnet50_fpn_v2": models.detection.maskrcnn_resnet50_fpn_v2(weights=models.detection.MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1),
}

keypoint_detection_models = {
    "keypointrcnn_resnet50_fpn": models.detection.keypointrcnn_resnet50_fpn(weights=models.detection.KeypointRCNN_ResNet50_FPN_Weights.COCO_V1),
}

## Facer